# Librería Pandas (agrupar, calculos)

_____________________________________________

In [1]:
import pandas as pd
import sqlite3
from datetime import datetime

In [2]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
ruta = 'D:\\Python\\Espacio\\Espacio.db'

bd = sqlite3.connect(ruta)
sql = 'SELECT * FROM unidades'
unidades = pd.read_sql_query(sql, bd)
sql = 'SELECT * FROM directorios'
directorio = pd.read_sql_query(sql, bd)
sql = 'SELECT * FROM ficheros'
fichero = pd.read_sql_query(sql, bd)

In [4]:
#  Tomamos SOLO los datos del NAS para pruebas
ficheros = fichero[fichero['volumen'] == 'PC_JAVI_1_DATA']
directorios = directorio[directorio['volumen'] == 'PC_JAVI_1_DATA']

In [5]:
#  Modificamos los datos de la BD Unidades 
#   - Añadimos columna Gb. de: total / usado / libre
#   - Reordenamos las columnas
#   - Formateamos la fecha
unidades['Gb'] = unidades['total'].apply(lambda n: n / 1024**3)
unidades['Gb.'] = unidades['usado'].apply(lambda n: n / 1024**3)
unidades['Gb..'] = unidades['libre'].apply(lambda n: n / 1024**3)
unidades = unidades.reindex(columns=['fecha', 'equipo', 'volumen', 'tipo', 'total', 'Gb', 'usado', 'Gb.', 'porcentje', 
                                     'libre', 'Gb..', 'tiempo'])
unidades['fecha'] = pd.to_datetime(unidades['fecha'], format="%Y-%m-%d %H:%M:%S").dt.strftime('%d-%m-%Y')
unidades

,fecha,equipo,volumen,tipo,total,Gb,usado,Gb.,porcentje,libre,Gb..,tiempo
0,04-06-2023,PC-JAVI,PC_JAVI_1_DATA,NTFS,658967097344,613.710934,106400108544,99.092823,NaN,552566988800,514.618111,"Tiempo lectura D:\ 0'4"""
1,04-06-2023,PC-JAVI,PC_JAVI_0_SISTEMA,NTFS,363752177664,338.770615,168834748416,157.239613,NaN,194917429248,181.531002,"Tiempo lectura C:\ 0'46"""
2,04-06-2023,NAS,NAS,BTRFS,0,0.000000,0,0.000000,NaN,0,0.000000,"Tiempo lectura NAS 1'23"""


In [6]:
#  Modificamos los datos de la BD Directorios 
#   - Añadimos columna Dir con el nivel de directorio (contando el núm de veces que tiene '\' + 1)
#   - Añadimos columna Gb. de tamaño
#   - Reordenamos las columnas
#   - Formateamos la fecha
directorios['Niv'] = directorios['ruta'].apply(lambda n: n.count('\\') + 1)
directorios['fecha'] = pd.to_datetime(directorios['fecha'], format="%Y-%m-%d %H:%M:%S").dt.strftime('%d-%m-%Y')
directorios['Gb'] = directorios['tamaño'].apply(lambda n: n / 1024**3)
directorios = directorios.reindex(columns=['fecha', 'equipo', 'volumen', 'Niv', 'ruta', 'tamaño', 'Gb', 'fechamod', 'numdir', 'numfich', 'error'])
directorios[directorios['Niv'] == 1]

,fecha,equipo,volumen,Niv,ruta,tamaño,Gb,fechamod,numdir,numfich,error
4,04-06-2023,PC-JAVI,PC_JAVI_1_DATA,1,$RECYCLE.BIN,261136913,0.243203,0001-01-01 00:00:00,2,7,
256,04-06-2023,PC-JAVI,PC_JAVI_1_DATA,1,App,51597738283,48.054138,2022-11-25 08:11:08,177,488,
269,04-06-2023,PC-JAVI,PC_JAVI_1_DATA,1,Backup,32058,0.000030,2023-05-15 11:06:17,3,24,
1898,04-06-2023,PC-JAVI,PC_JAVI_1_DATA,1,Documentos,6825951835,6.357163,2023-06-02 09:57:54,308,4232,
2875,04-06-2023,PC-JAVI,PC_JAVI_1_DATA,1,Informatica,41384155709,38.541998,2023-05-30 09:13:26,460,1627,
5796,04-06-2023,PC-JAVI,PC_JAVI_1_DATA,1,Python,3582048347,3.336042,2023-05-26 07:31:27,1476,9289,
5797,04-06-2023,PC-JAVI,PC_JAVI_1_DATA,1,System Volume Information,0,0.000000,0001-01-01 00:00:00,1,0,Acceso Denegado1
5818,04-06-2023,PC-JAVI,PC_JAVI_1_DATA,1,Temp,2429837713,2.262963,2023-04-27 07:49:58,14,108,
5819,04-06-2023,PC-JAVI,PC_JAVI_1_DATA,1,,106080900858,98.795538,0001-01-01 00:00:00,2442,15775,


In [7]:
#  Contar las veces que aparece un fichero, lo primero es obtener el nombre del fichero juntando nombre y extension
ficheros['fichero'] = ficheros['nombre'] + ficheros['extension']
ficheros['contar'] = ficheros.groupby('fichero')['fichero'].transform('count')
ficheros['fecha'] = pd.to_datetime(ficheros['fecha'], format="%Y-%m-%d %H:%M:%S").dt.strftime('%d-%m-%Y')
ficheros['fechamod'] = pd.to_datetime(ficheros['fechamod'], format="%Y-%m-%d %H:%M:%S").dt.strftime('%d-%m-%Y %H:%M')
ficheros['Gb'] = ficheros['tamaño'].apply(lambda n: n / 1024**3)
del ficheros['fichero']
ficheros = ficheros.reindex(columns=['fecha', 'equipo', 'volumen', 'ruta', 'nombre', 'extension', 'tipo', 'tamaño', 'Gb', 'fechamod', 'contar', 'error'])

print("Ficheros repetidos + de 2 veces")
ficheros[ficheros['contar'] == 2]

Ficheros repetidos + de 2 veces


,fecha,equipo,volumen,ruta,nombre,extension,tipo,tamaño,Gb,fechamod,contar,error
8,04-06-2023,PC-JAVI,PC_JAVI_1_DATA,$RECYCLE.BIN\S-1-5-21-1460955175-1052792528-18...,desktop,.ini,,129,1.201406e-07,24-04-2023 17:41,2,
55,04-06-2023,PC-JAVI,PC_JAVI_1_DATA,App\$ Sin Clasificar\$ Nuevos\Derive v6,Instrucciones,.txt,Texto,450,4.190952e-07,09-05-2008 21:16,2,
95,04-06-2023,PC-JAVI,PC_JAVI_1_DATA,App\$ Sin Clasificar\$ Utilidades USB\USB Deview,readme,.txt,Texto,16694,1.554750e-05,13-01-2009 22:29,2,
107,04-06-2023,PC-JAVI,PC_JAVI_1_DATA,App\$ Sin Clasificar,AntiTwin,.rar,,763204,7.107891e-04,15-12-2009 21:08,2,
135,04-06-2023,PC-JAVI,PC_JAVI_1_DATA,App\$ Sin Clasificar,CloneDVD v2.8.9.9,.rar,,5352753,4.985140e-03,14-12-2009 20:54,2,
...,...,...,...,...,...,...,...,...,...,...,...,...
44315,04-06-2023,PC-JAVI,PC_JAVI_1_DATA,Python\Guía_Python\Módulos\Ficheros,football_noticias,.csv,Excel,9414,8.767471e-06,29-09-2022 08:00,2,
44525,04-06-2023,PC-JAVI,PC_JAVI_1_DATA,Python\WebScrapping\Ficheros,Indexa Pensión,.txt,Texto,0,0.000000e+00,20-10-2022 07:03,2,
44674,04-06-2023,PC-JAVI,PC_JAVI_1_DATA,Temp,2023-02 Concurso Primavera Matemáticas 1 y 2 E...,.pdf,Pdf,831231,7.741442e-04,16-02-2023 13:03,2,
44676,04-06-2023,PC-JAVI,PC_JAVI_1_DATA,Temp,2023-02 Concurso Primavera Matemáticas 3 y 4 E...,.pdf,Pdf,1007120,9.379536e-04,16-02-2023 13:04,2,


#### PRUEBAS PARA SACAR EL TAMAÑO DEL DIRECTORIO 
- Sumar el tamaño de los ficheros del volumen - ruta

### Ejemplo de agrupar y devolver en columna

In [62]:
df = pd.DataFrame({'lista':list('abssbab')})
#  Añadimos una nueva columna
df['nueva'] = 4
print(df)
#  Contamos valores únicos de columna lista
print("value_counts - valores únicos\n", df['lista'].value_counts())
#  Uso de groupby
df1 = df.groupby('lista').count()
print("Uso de groupby\n", df1)
#  Uso de groupby y 
df['freq'] = df.groupby('lista')['lista'].transform('count')
print("\nUso de groupby y transform para incluir en columna\n", df)

  lista  nueva
0     a      4
1     b      4
2     s      4
3     s      4
4     b      4
5     a      4
6     b      4
value_counts - valores únicos
 lista
b    3
a    2
s    2
Name: count, dtype: int64
Uso de groupby
        nueva
lista       
a          2
b          3
s          2

Uso de groupby y transform para incluir en columna
   lista  nueva  freq
0     a      4     2
1     b      4     3
2     s      4     2
3     s      4     2
4     b      4     3
5     a      4     2
6     b      4     3
